In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor, VotingRegressor
from sklearn import linear_model 
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from xgboost import XGBRegressor
from scipy.stats import randint, uniform
from sklearn.linear_model import LinearRegression, Ridge, Lasso


In [30]:
sample_solution = pd.read_csv("133e814757ed11f0/dataset/sample_solution.csv", index_col='ID')

In [31]:
sample_solution.columns

Index(['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],
      dtype='object')

In [32]:
sample_solution

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
ID,,,,,,,,,,
1,-0.117370,0.348090,0.473673,0.079501,-0.411504,0.015352,0.454957,0.065651,-0.146684,-0.140500
2,-0.503910,-0.250186,-1.412727,-0.523577,-0.577571,-0.294264,-1.396187,-0.856044,-0.003241,-0.246948
3,1.484172,1.272972,1.188539,1.321349,1.472491,1.237584,1.192748,1.575889,0.773926,1.917254
4,0.841616,0.457436,0.534375,0.376650,1.593406,0.157950,0.516430,0.632370,0.376289,-0.446052
5,-0.024147,0.136198,1.174866,-0.197264,2.463520,0.418315,1.185596,0.509797,-0.434762,0.807128
...,...,...,...,...,...,...,...,...,...,...
496,0.022706,-0.047897,0.474691,-0.016551,-0.431127,-0.081999,0.462777,0.089287,-0.608682,0.516307
497,-1.475840,-1.294104,-1.230311,-1.231267,-0.303306,-0.977209,-1.217209,-1.553847,-1.126938,-1.407277
498,1.002035,0.870014,0.037003,0.963333,-0.261270,0.620495,0.029488,0.414339,0.726121,-0.088445


In [33]:
train_dataset = pd.read_csv('133e814757ed11f0/dataset/train.csv')
test_dataset  = pd.read_csv('133e814757ed11f0/dataset/test.csv',index_col='ID')

In [34]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 65 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    2000 non-null   float64
 1   Component2_fraction    2000 non-null   float64
 2   Component3_fraction    2000 non-null   float64
 3   Component4_fraction    2000 non-null   float64
 4   Component5_fraction    2000 non-null   float64
 5   Component1_Property1   2000 non-null   float64
 6   Component2_Property1   2000 non-null   float64
 7   Component3_Property1   2000 non-null   float64
 8   Component4_Property1   2000 non-null   float64
 9   Component5_Property1   2000 non-null   float64
 10  Component1_Property2   2000 non-null   float64
 11  Component2_Property2   2000 non-null   float64
 12  Component3_Property2   2000 non-null   float64
 13  Component4_Property2   2000 non-null   float64
 14  Component5_Property2   2000 non-null   float64
 15  Comp

In [35]:
train_dataset, output_blends = train_dataset.iloc[:,:55],train_dataset.iloc[:,55:]

In [36]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1 to 500
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Component1_fraction    500 non-null    float64
 1   Component2_fraction    500 non-null    float64
 2   Component3_fraction    500 non-null    float64
 3   Component4_fraction    500 non-null    float64
 4   Component5_fraction    500 non-null    float64
 5   Component1_Property1   500 non-null    float64
 6   Component2_Property1   500 non-null    float64
 7   Component3_Property1   500 non-null    float64
 8   Component4_Property1   500 non-null    float64
 9   Component5_Property1   500 non-null    float64
 10  Component1_Property2   500 non-null    float64
 11  Component2_Property2   500 non-null    float64
 12  Component3_Property2   500 non-null    float64
 13  Component4_Property2   500 non-null    float64
 14  Component5_Property2   500 non-null    float64
 15  Component1_

In [37]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component1_Property9,Component2_Property9,Component3_Property9,Component4_Property9,Component5_Property9,Component1_Property10,Component2_Property10,Component3_Property10,Component4_Property10,Component5_Property10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,0.480368,1.044967,-0.450956,0.674572,-0.636394,-1.244963,-1.355050,-0.314423,0.993593,-2.728928
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,-1.958826,-0.019603,-0.807923,0.148715,1.439313,-1.160435,-0.014276,-0.135968,-1.221155,0.896222
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,-0.798978,-0.444027,0.148405,-0.793607,0.123834,0.006829,0.668734,0.015449,-0.098661,-0.424314
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-0.534135,1.155513,-0.760428,0.450159,-0.973779,0.052972,-1.024785,0.118951,2.400556,-0.576430
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.389350,1.799238,-0.912374,1.767557,-0.467038,2.104922,0.858593,-0.469110,0.715789,-2.038341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,1.138839,1.666804,-1.413339,0.405253,0.766653,-0.322096,1.399468,1.096369,-0.346225,0.641193
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.782418,0.784366,1.113626,1.328112,-2.537512,0.461525,0.647984,-0.618766,-0.047918,0.397253
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,-0.813747,-0.197880,-0.549162,0.810814,1.567580,-0.694918,-1.710215,-0.233936,-0.133002,-0.284672
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,1.262477,-0.925444,-0.823345,0.427648,-0.161447,0.628131,-0.038484,0.343058,0.448748,0.193507


In [38]:
output_blends

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,0.489143,0.607589,0.321670,-1.236055,1.601132,1.384662,0.305850,0.193460,0.580374,-0.762738
1,-1.257481,-1.475283,-0.437385,-1.402911,0.147941,-1.143244,-0.439171,-1.379041,-1.280989,-0.503625
2,1.784349,0.450467,0.622687,1.375614,-0.428790,1.161616,0.601289,0.872950,0.660000,2.024576
3,-0.066422,0.483730,-1.865442,-0.046295,-0.163820,-0.209693,-1.840566,0.300293,-0.351336,-1.551914
4,-0.118913,-1.172398,0.301785,-1.787407,-0.493361,-0.528049,0.286344,-0.265192,0.430513,0.735073
...,...,...,...,...,...,...,...,...,...,...
1995,-0.028366,-0.327297,-0.316933,-1.294092,-0.530259,-0.421526,-0.320869,0.709627,-0.737244,-0.744289
1996,-0.449245,0.156778,-0.367445,-0.938615,-0.577451,-0.209996,-0.370505,-0.195531,-0.032834,0.269718
1997,0.029135,0.164890,-0.092942,-1.134490,-0.437479,-0.695636,-0.101073,0.063650,0.624368,-0.477053
1998,-0.232960,-0.464947,0.112536,-0.793522,-0.811272,-1.194914,0.100644,0.760116,-0.751394,-0.857598


In [39]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# ------------------------------------------------------------------
# 0.  LOAD & COPY ORIGINAL DATAFRAME
# ------------------------------------------------------------------
# assume your raw dataframe is named `df`
# train_dataset = df.copy()

# ------------------------------------------------------------------
# 1.  COLUMN SET‑UP
# ------------------------------------------------------------------
fraction_cols = [f'Component{i}_fraction' for i in range(1, 6)]

property_cols = {}
for prop_idx in range(1, 11):
    property_cols[prop_idx] = [
        f'Component{i}_Property{prop_idx}' for i in range(1, 6)
    ]

# ------------------------------------------------------------------
# 2.  LINEAR BLEND FEATURES
# ------------------------------------------------------------------
for prop_idx in range(1, 11):
    w_col = f'Weighted_Prop{prop_idx}'
    train_dataset[w_col] = 0.0
    for comp_idx in range(1, 6):
        train_dataset[w_col] += (
            train_dataset[f'Component{comp_idx}_fraction']
            * train_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )
    for comp_idx in range(1, 6):
        train_dataset[f'Deviation_{comp_idx}_Prop{prop_idx}'] = (
            train_dataset[f'Component{comp_idx}_Property{prop_idx}']
            - train_dataset[w_col]
        )

# ------------------------------------------------------------------
# 3.  FRACTION‑BASED FEATURES
# ------------------------------------------------------------------
train_dataset['Fraction_Entropy'] = (
    -train_dataset[fraction_cols] * np.log(train_dataset[fraction_cols] + 1e-10)
).sum(axis=1)
train_dataset['Dominant_Fraction'] = train_dataset[fraction_cols].max(axis=1)
train_dataset['Fraction_Range']    = (
    train_dataset[fraction_cols].max(axis=1)
    - train_dataset[fraction_cols].min(axis=1)
)

for i in range(1, 6):
    for j in range(i + 1, 6):
        train_dataset[f'Frac_Interaction_{i}_{j}'] = (
            train_dataset[f'Component{i}_fraction']
            * train_dataset[f'Component{j}_fraction']
        )

# ------------------------------------------------------------------
# 4.  PROPERTY INTERACTION FEATURES
# ------------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    train_dataset[f'Prop{prop_idx}_Min']      = train_dataset[cols].min(axis=1)
    train_dataset[f'Prop{prop_idx}_Max']      = train_dataset[cols].max(axis=1)
    train_dataset[f'Prop{prop_idx}_Range']    = (
        train_dataset[f'Prop{prop_idx}_Max'] - train_dataset[f'Prop{prop_idx}_Min']
    )
    train_dataset[f'Prop{prop_idx}_Variance'] = train_dataset[cols].var(axis=1)

    max_vals = train_dataset[cols].max(axis=1)
    for comp_idx in range(1, 6):
        train_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
            train_dataset[cols[comp_idx - 1]] == max_vals
        ).astype(int)

# ------------------------------------------------------------------
# 5.  CROSS‑PROPERTY INTERACTIONS
# ------------------------------------------------------------------
for comp_idx in range(1, 6):
    comp_props = [f'Component{comp_idx}_Property{p}' for p in range(1, 11)]
    train_dataset[f'Comp{comp_idx}_Prop_Mean'] = train_dataset[comp_props].mean(axis=1)
    train_dataset[f'Comp{comp_idx}_Prop_Std']  = train_dataset[comp_props].std(axis=1)

    for prop_idx in range(1, 11):
        train_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Scaled'] = (
            train_dataset[f'Component{comp_idx}_fraction']
            * train_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )

# ------------------------------------------------------------------
# 6.  NON‑LINEAR TRANSFORMATIONS
# ------------------------------------------------------------------
for comp_idx in range(1, 6):
    frac_col = f'Component{comp_idx}_fraction'
    train_dataset[f'{frac_col}_Sq']   = train_dataset[frac_col] ** 2
    train_dataset[f'{frac_col}_Sqrt'] = np.sqrt(train_dataset[frac_col])

    for prop_idx in range(1, 11):
        col = f'Component{comp_idx}_Property{prop_idx}'
        train_dataset[f'{col}_Log'] = np.log1p(train_dataset[col])
        train_dataset[f'{col}_Sq']  = train_dataset[col] ** 2

# ------------------------------------------------------------------
# 7.  MIXTURE CHARACTERISTICS
# ------------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    max_val = train_dataset[cols].max(axis=1)
    min_val = train_dataset[cols].min(axis=1)
    train_dataset[f'Prop{prop_idx}_BlendRatio'] = (
        (max_val - min_val) / (max_val + 1e-10)
    )

    for comp_idx in range(1, 6):
        col = f'Component{comp_idx}_Property{prop_idx}'
        train_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Adj'] = (
            train_dataset[col] * (1 + train_dataset[f'Component{comp_idx}_fraction'])
        )

# ------------------------------------------------------------------
# 8.  CLUSTER LABEL (optional, no target leak)
# ------------------------------------------------------------------
cluster_features = [f'Weighted_Prop{i}' for i in range(1, 11)] + fraction_cols
kmeans = KMeans(n_clusters=5, random_state=42)
# fit on train
_ = kmeans.fit(train_dataset[cluster_features])

# train: compute distances to each center
train_dists = kmeans.transform(train_dataset[cluster_features])
train_dist_df = pd.DataFrame(
    train_dists,
    columns=[f'Cluster_Dist_{i}' for i in range(5)],
    index=train_dataset.index
)
train_dataset = pd.concat([train_dataset, train_dist_df], axis=1)
# ------------------------------------------------------------------
# 9.  TARGET‑INDEPENDENT CONTRIBUTIONS
# ------------------------------------------------------------------
for target_idx in range(1, 11):
    w_col = f'Weighted_Prop{target_idx}'
    for comp_idx in range(1, 6):
        contrib_col = f'Comp{comp_idx}_Contrib_Prop{target_idx}'
        train_dataset[contrib_col] = (
            train_dataset[f'Component{comp_idx}_fraction']
            * train_dataset[f'Component{comp_idx}_Property{target_idx}']
            / (train_dataset[w_col] + 1e-10)
        )

# ------------------------------------------------------------------
# 10.  CLEAN‑UP
# ------------------------------------------------------------------
train_dataset.drop(columns=['Blend_Cluster'], inplace=True)


/tmp/ipykernel_4926/4276239847.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
/tmp/ipykernel_4926/4276239847.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_dataset[f'Prop{prop_idx}_Min']      = train_dataset[cols].min(axis=1)
/tmp/ipykernel_4926/4276239847.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

KeyError: "['Blend_Cluster'] not found in axis"

In [40]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Comp1_Contrib_Prop9,Comp2_Contrib_Prop9,Comp3_Contrib_Prop9,Comp4_Contrib_Prop9,Comp5_Contrib_Prop9,Comp1_Contrib_Prop10,Comp2_Contrib_Prop10,Comp3_Contrib_Prop10,Comp4_Contrib_Prop10,Comp5_Contrib_Prop10
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,26.889721,0.000000,-50.486585,44.953230,-20.356366,0.553231,0.000000,0.279444,-0.525630,0.692954
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,0.299647,0.049479,1.174108,-0.523234,-0.000000,0.037708,0.007654,0.041973,0.912665,-0.000000
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,0.142130,0.078988,-0.059399,0.882339,-0.044057,-0.009046,-0.885818,-0.046045,0.816799,1.124109
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,-1.026313,3.730034,-0.000000,0.242188,-1.945908,-0.033191,1.078722,-0.000000,-0.421151,0.375619
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,-0.101491,0.288618,-0.073178,0.886051,-0.000000,0.544546,0.136689,-0.037341,0.356107,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,0.588973,0.206885,-0.000000,0.108984,0.095158,26.030345,-27.143699,-0.000000,14.549770,-12.436416
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.580659,0.949748,0.000000,1.919396,-1.288486,0.271925,0.622911,-0.000000,-0.054979,0.160144
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,-11.982363,-0.460069,-2.979187,9.739846,6.681772,0.559495,0.217411,0.069391,0.087357,0.066346
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,1.181403,-0.277124,-0.000000,0.144067,-0.048345,0.748410,-0.014673,0.000000,0.192484,0.073780


In [41]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# ------------------------------------------------------------
# 0. COPY RAW TEST DATA
# ------------------------------------------------------------
# assume your test dataframe is named `test_df`
# test_dataset = test_df.copy()

# ------------------------------------------------------------
# 1. COLUMN SETUP
# ------------------------------------------------------------
fraction_cols = [f'Component{i}_fraction' for i in range(1, 6)]
property_cols = {
    prop_idx: [f'Component{i}_Property{prop_idx}' for i in range(1, 6)]
    for prop_idx in range(1, 11)
}

# ------------------------------------------------------------
# 2. LINEAR BLEND FEATURES
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    w_col = f'Weighted_Prop{prop_idx}'
    test_dataset[w_col] = 0.0
    for comp_idx in range(1, 6):
        test_dataset[w_col] += (
            test_dataset[f'Component{comp_idx}_fraction']
            * test_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )
    for comp_idx in range(1, 6):
        test_dataset[f'Deviation_{comp_idx}_Prop{prop_idx}'] = (
            test_dataset[f'Component{comp_idx}_Property{prop_idx}']
            - test_dataset[w_col]
        )

# ------------------------------------------------------------
# 3. FRACTION-BASED FEATURES
# ------------------------------------------------------------
test_dataset['Fraction_Entropy'] = (
    -test_dataset[fraction_cols] * np.log(test_dataset[fraction_cols] + 1e-10)
).sum(axis=1)
test_dataset['Dominant_Fraction'] = test_dataset[fraction_cols].max(axis=1)
test_dataset['Fraction_Range']    = (
    test_dataset[fraction_cols].max(axis=1)
    - test_dataset[fraction_cols].min(axis=1)
)

for i in range(1, 6):
    for j in range(i + 1, 6):
        test_dataset[f'Frac_Interaction_{i}_{j}'] = (
            test_dataset[f'Component{i}_fraction']
            * test_dataset[f'Component{j}_fraction']
        )

# ------------------------------------------------------------
# 4. PROPERTY INTERACTION FEATURES
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    test_dataset[f'Prop{prop_idx}_Min']      = test_dataset[cols].min(axis=1)
    test_dataset[f'Prop{prop_idx}_Max']      = test_dataset[cols].max(axis=1)
    test_dataset[f'Prop{prop_idx}_Range']    = (
        test_dataset[f'Prop{prop_idx}_Max'] - test_dataset[f'Prop{prop_idx}_Min']
    )
    test_dataset[f'Prop{prop_idx}_Variance'] = test_dataset[cols].var(axis=1)

    max_vals = test_dataset[cols].max(axis=1)
    for comp_idx in range(1, 6):
        test_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
            test_dataset[cols[comp_idx - 1]] == max_vals
        ).astype(int)

# ------------------------------------------------------------
# 5. CROSS-PROPERTY INTERACTIONS
# ------------------------------------------------------------
for comp_idx in range(1, 6):
    comp_props = [f'Component{comp_idx}_Property{p}' for p in range(1, 11)]
    test_dataset[f'Comp{comp_idx}_Prop_Mean'] = test_dataset[comp_props].mean(axis=1)
    test_dataset[f'Comp{comp_idx}_Prop_Std']  = test_dataset[comp_props].std(axis=1)

    for prop_idx in range(1, 11):
        test_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Scaled'] = (
            test_dataset[f'Component{comp_idx}_fraction']
            * test_dataset[f'Component{comp_idx}_Property{prop_idx}']
        )

# ------------------------------------------------------------
# 6. NON-LINEAR TRANSFORMATIONS
# ------------------------------------------------------------
for comp_idx in range(1, 6):
    frac_col = f'Component{comp_idx}_fraction'
    test_dataset[f'{frac_col}_Sq']   = test_dataset[frac_col] ** 2
    test_dataset[f'{frac_col}_Sqrt'] = np.sqrt(test_dataset[frac_col])

    for prop_idx in range(1, 11):
        col = f'Component{comp_idx}_Property{prop_idx}'
        test_dataset[f'{col}_Log'] = np.log1p(test_dataset[col])
        test_dataset[f'{col}_Sq']  = test_dataset[col] ** 2

# ------------------------------------------------------------
# 7. MIXTURE CHARACTERISTICS
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    cols = property_cols[prop_idx]
    max_val = test_dataset[cols].max(axis=1)
    min_val = test_dataset[cols].min(axis=1)
    test_dataset[f'Prop{prop_idx}_BlendRatio'] = (
        (max_val - min_val) / (max_val + 1e-10)
    )

    for comp_idx in range(1, 6):
        col = f'Component{comp_idx}_Property{prop_idx}'
        test_dataset[f'Comp{comp_idx}_Prop{prop_idx}_Adj'] = (
            test_dataset[col] * (1 + test_dataset[f'Component{comp_idx}_fraction'])
        )

# ------------------------------------------------------------
# 8. OPTIONAL: CLUSTER LABEL (no target leak)
# ------------------------------------------------------------
cluster_features = [f'Weighted_Prop{i}' for i in range(1, 11)] + fraction_cols

# test: compute distances
test_dists = kmeans.transform(test_dataset[cluster_features])
test_dist_df = pd.DataFrame(
    test_dists,
    columns=[f'Cluster_Dist_{i}' for i in range(5)],
    index=test_dataset.index
)
test_dataset = pd.concat([test_dataset, test_dist_df], axis=1)

# ------------------------------------------------------------
# 9. TARGET-INDEPENDENT CONTRIBUTIONS
# ------------------------------------------------------------
for prop_idx in range(1, 11):
    w_col = f'Weighted_Prop{prop_idx}'
    for comp_idx in range(1, 6):
        contrib_col = f'Comp{comp_idx}_Contrib_Prop{prop_idx}'
        test_dataset[contrib_col] = (
            test_dataset[f'Component{comp_idx}_fraction']
            * test_dataset[f'Component{comp_idx}_Property{prop_idx}']
            / (test_dataset[w_col] + 1e-10)
        )

# ------------------------------------------------------------
# 10. CLEAN-UP
# ------------------------------------------------------------
test_dataset.drop(columns=['Blend_Cluster'], inplace=True)


/tmp/ipykernel_4926/4065723515.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_dataset[f'IsMax_Comp{comp_idx}_Prop{prop_idx}'] = (
/tmp/ipykernel_4926/4065723515.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_dataset[f'Prop{prop_idx}_Min']      = test_dataset[cols].min(axis=1)
/tmp/ipykernel_4926/4065723515.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conc

KeyError: "['Blend_Cluster'] not found in axis"

#  Add More Featurs

In [42]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Comp1_Contrib_Prop9,Comp2_Contrib_Prop9,Comp3_Contrib_Prop9,Comp4_Contrib_Prop9,Comp5_Contrib_Prop9,Comp1_Contrib_Prop10,Comp2_Contrib_Prop10,Comp3_Contrib_Prop10,Comp4_Contrib_Prop10,Comp5_Contrib_Prop10
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,-6.613360,0.854447,1.264097,-8.880794,14.375610,-5.896530,1.596623,-20.281949,19.282980,6.298877
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,0.000000,0.365887,0.000000,0.702329,-0.068216,-0.000000,1.136398,-0.000000,0.271652,-0.408050
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,0.248486,0.000000,-0.325015,1.111306,-0.034776,-0.709326,0.000000,-0.256346,1.838264,0.127407
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,0.516786,-0.000000,0.024645,0.216259,0.242310,2.080386,-0.000000,0.002130,1.322353,-2.404868
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,0.000000,0.000000,1.973529,-0.973529,0.000000,-0.000000,0.000000,-2.617826,3.617826,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,0.685804,0.001340,0.063311,0.202802,0.046744,0.780083,0.009782,0.030665,0.225261,-0.045791
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,0.118870,0.912187,-0.079909,0.088940,-0.040088,0.529741,0.827689,-0.010229,-0.209668,-0.137534
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,0.190230,0.008189,0.112344,0.314207,0.375031,1.473952,0.033349,0.165223,-0.655451,-0.017072


In [43]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

def create_blending_features(df):
    # Create a copy to avoid modifying original data
    df = df.copy()
    
    # Assumptions about property indices (based on petroleum blending domain knowledge):
    # Property1: API Gravity (density measure)
    # Property2: RVP (Reid Vapor Pressure)
    # Property3: RON (Research Octane Number)
    # Property4: Olefins content
    # Property5: Aromatics content
    # Property6: T10 (10% distillation temperature)
    # Property7: T50 (50% distillation temperature)
    # Property8: T90 (90% distillation temperature)
    # Property9: Viscosity
    # Property10: Cetane Index
    
    # 1. Density and Gravity Features
    for i in range(1, 6):
        # Convert API to Specific Gravity (60/60)
        df[f'Component{i}_SG'] = 141.5 / (df[f'Component{i}_Property1'] + 131.5)
        
        # Calculate density blending index
        df[f'Component{i}_Density_Index'] = df[f'Component{i}_SG'] ** (-0.06)
    
    # Blend density index volumetrically
    df['Blend_Density_Index'] = 0
    for i in range(1, 6):
        df['Blend_Density_Index'] += df[f'Component{i}_fraction'] * df[f'Component{i}_Density_Index']
    
    # Convert back to specific gravity
    df['Blend_SG'] = df['Blend_Density_Index'] ** (-1/0.06)
    df['Blend_API'] = (141.5 / df['Blend_SG']) - 131.5
    
    # 2. Sulfur Blending (gravimetric) - assuming Property2 is sulfur-related
    df['Blend_Sulfur'] = 0
    for i in range(1, 6):
        df['Blend_Sulfur'] += (df[f'Component{i}_fraction'] * df[f'Component{i}_SG'] * 
                              df[f'Component{i}_Property2']) / df['Blend_SG']
    
    # 3. RVP Blending (Index Method)
    rvp_exponent = -0.07  # Texaco method exponent
    for i in range(1, 6):
        df[f'Component{i}_RVP_Index'] = df[f'Component{i}_Property2'] ** rvp_exponent
    
    df['Blend_RVP_Index'] = 0
    for i in range(1, 6):
        df['Blend_RVP_Index'] += df[f'Component{i}_fraction'] * df[f'Component{i}_RVP_Index']
    
    df['Blend_RVP'] = df['Blend_RVP_Index'] ** (1/rvp_exponent)
    
    # 4. Octane Blending (Ethyl RT-70 method)
    for i in range(1, 6):
        # Simplified octane interaction term
        df[f'Component{i}_RON_BlendValue'] = (
            df[f'Component{i}_Property3'] + 
            0.1 * df[f'Component{i}_Property4'] - 
            0.05 * df[f'Component{i}_Property5']
        )
    
    df['Blend_RON'] = 0
    for i in range(1, 6):
        df['Blend_RON'] += df[f'Component{i}_fraction'] * df[f'Component{i}_RON_BlendValue']
    
    # 5. Distillation Blending (Ethyl S-Curve Model)
    distillation_points = {'T10': 6, 'T50': 7, 'T90': 8}
    for point, prop_idx in distillation_points.items():
        for i in range(1, 6):
            # Simplified distillation blending index
            df[f'Component{i}_{point}_Index'] = np.log(df[f'Component{i}_Property{prop_idx}'])
        
        df[f'Blend_{point}_Index'] = 0
        for i in range(1, 6):
            df[f'Blend_{point}_Index'] += (
                df[f'Component{i}_fraction'] * 
                df[f'Component{i}_{point}_Index']
            )
        
        df[f'Blend_{point}'] = np.exp(df[f'Blend_{point}_Index'])
    
    # 6. Cetane Index (ASTM D-976)
    for i in range(1, 6):
        df[f'Component{i}_Cetane_Index'] = (
            420.34 + 0.016 * df[f'Component{i}_Property1']**2 + 
            0.192 * df[f'Component{i}_Property1'] * np.log(df[f'Component{i}_Property7']) + 
            65.01 * (np.log(df[f'Component{i}_Property7']))**2 - 
            0.0001809 * df[f'Component{i}_Property7']**2
        )
    
    df['Blend_Cetane'] = 0
    for i in range(1, 6):
        df['Blend_Cetane'] += df[f'Component{i}_fraction'] * df[f'Component{i}_Cetane_Index']
    
    # 7. Viscosity Blending
    for i in range(1, 6):
        df[f'Component{i}_Visc_Index'] = np.log(np.log(df[f'Component{i}_Property9'] + 0.8))
    
    df['Blend_Visc_Index'] = 0
    for i in range(1, 6):
        df['Blend_Visc_Index'] += df[f'Component{i}_fraction'] * df[f'Component{i}_Visc_Index']
    
    df['Blend_Viscosity'] = np.exp(np.exp(df['Blend_Visc_Index'])) - 0.8
    
    # # 8. Interaction Terms
    # for i in range(1, 6):
    #     for j in range(i+1, 6):
    #         df[f'Frac_Interaction_{i}_{j}'] = (
    #             df[f'Component{i}_fraction'] * 
    #             df[f'Component{j}_fraction']
    #         )
    
    # 9. Polynomial Features
    # poly = PolynomialFeatures(degree=2, include_bias=False)
    # component_features = [f'Component{i}_fraction' for i in range(1, 6)] + \
    #                      [f'Component{i}_Property1' for i in range(1, 6)] + \
    #                      [f'Component{i}_Property2' for i in range(1, 6)]
    
    # poly_features = poly.fit_transform(df[component_features])
    # poly_cols = [f'Poly_{i}' for i in range(poly_features.shape[1])]
    # poly_df = pd.DataFrame(poly_features, columns=poly_cols)
    # df = pd.concat([df, poly_df], axis=1)
    
    # 10. Mixture Complexity Metrics
    fractions = [f'Component{i}_fraction' for i in range(1, 6)]
    df['Fraction_Entropy'] = (-df[fractions] * np.log(df[fractions] + 1e-9)).sum(axis=1)
    df['Dominant_Fraction'] = df[fractions].max(axis=1)
    
    # 11. Final Cleaning
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(), inplace=True)
    
    return df

# Apply feature engineering to both datasets
train_dataset = create_blending_features(train_dataset)
test_dataset = create_blending_features(test_dataset)

/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/harshit/m

In [44]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,686.546409,567.997235,504.410185,-1.397769,-0.490269,-1.013895,-0.945803,-0.975395,-1.117089,0.681842
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,485.796592,701.435666,504.410185,-0.979237,-0.955023,-1.013895,-1.020018,-0.215462,-1.117089,0.681842
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,519.163309,534.327773,504.410185,-0.979237,-0.955023,-1.013895,-1.020018,-0.975395,-1.117089,0.681842
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,420.665290,534.327773,504.410185,-0.979237,-0.399504,-1.013895,-1.499369,-0.975395,-1.117089,0.681842
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,519.163309,450.234034,504.410185,-0.979237,-0.045815,-1.013895,-0.058737,-0.975395,-1.117089,0.681842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,423.640833,421.011697,504.410185,-0.412355,-0.102118,-1.013895,-1.678307,-0.800862,-1.117089,0.681842
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,519.163309,534.327773,504.410185,-0.979237,-0.776129,-0.432323,-0.280726,-0.975395,-1.117089,0.681842
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,519.163309,534.327773,504.410185,-0.979237,-0.955023,-1.013895,-0.740785,-0.148653,-1.117089,0.681842
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,446.859486,430.098069,427.511045,-0.323091,-0.955023,-1.013895,-1.584259,-0.975395,-1.117089,0.681842


In [45]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.299825,-1.303419,0.549224
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,430.534316,423.240164,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-1.219901,-1.303419,0.549224
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,533.739728,663.815825,476.367101,-1.055064,-0.845686,-0.663087,-1.111906,-0.970853,-1.303419,0.549224
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,533.739728,517.803098,476.367101,-1.055064,-2.890751,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,456.068496,424.246233,476.367101,-1.055064,-0.845686,-1.197669,-3.376006,-0.970853,-1.303419,0.549224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,422.428886,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,436.733566,517.803098,476.367101,-1.055064,-0.845686,0.012447,-1.111906,-1.866185,-1.303419,0.549224
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224


In [46]:
train_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,686.546409,567.997235,504.410185,-1.397769,-0.490269,-1.013895,-0.945803,-0.975395,-1.117089,0.681842
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,485.796592,701.435666,504.410185,-0.979237,-0.955023,-1.013895,-1.020018,-0.215462,-1.117089,0.681842
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,519.163309,534.327773,504.410185,-0.979237,-0.955023,-1.013895,-1.020018,-0.975395,-1.117089,0.681842
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,420.665290,534.327773,504.410185,-0.979237,-0.399504,-1.013895,-1.499369,-0.975395,-1.117089,0.681842
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,519.163309,450.234034,504.410185,-0.979237,-0.045815,-1.013895,-0.058737,-0.975395,-1.117089,0.681842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,423.640833,421.011697,504.410185,-0.412355,-0.102118,-1.013895,-1.678307,-0.800862,-1.117089,0.681842
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,519.163309,534.327773,504.410185,-0.979237,-0.776129,-0.432323,-0.280726,-0.975395,-1.117089,0.681842
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,519.163309,534.327773,504.410185,-0.979237,-0.955023,-1.013895,-0.740785,-0.148653,-1.117089,0.681842
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,446.859486,430.098069,427.511045,-0.323091,-0.955023,-1.013895,-1.584259,-0.975395,-1.117089,0.681842


In [47]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.299825,-1.303419,0.549224
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,430.534316,423.240164,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-1.219901,-1.303419,0.549224
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,533.739728,663.815825,476.367101,-1.055064,-0.845686,-0.663087,-1.111906,-0.970853,-1.303419,0.549224
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,533.739728,517.803098,476.367101,-1.055064,-2.890751,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,456.068496,424.246233,476.367101,-1.055064,-0.845686,-1.197669,-3.376006,-0.970853,-1.303419,0.549224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,422.428886,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,436.733566,517.803098,476.367101,-1.055064,-0.845686,0.012447,-1.111906,-1.866185,-1.303419,0.549224
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224


In [48]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 1 to 500
Columns: 564 entries, Component1_fraction to Blend_Viscosity
dtypes: float64(514), int64(50)
memory usage: 2.2 MB


# polynomial _Features creation

In [49]:
train_dataset.corr()

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
Component1_fraction,1.000000,-0.452466,-0.441056,-0.128280,0.028830,-0.015530,-0.032505,-0.000743,0.006344,0.042178,...,0.010787,-0.001266,0.025797,-0.004061,0.011581,0.006799,-0.016042,-0.038140,-0.017628,-0.014670
Component2_fraction,-0.452466,1.000000,-0.430003,-0.162097,0.056293,0.024858,0.020551,-0.039978,-0.007805,-0.025751,...,0.027715,0.013182,-0.014697,-0.000662,-0.017036,0.000670,0.015175,0.008998,0.005702,-0.004778
Component3_fraction,-0.441056,-0.430003,1.000000,-0.205391,0.063267,-0.024512,0.010782,0.030103,-0.001353,-0.005236,...,-0.041647,0.006413,0.001988,-0.004948,0.008847,-0.012119,-0.005723,0.020535,-0.005795,0.006216
Component4_fraction,-0.128280,-0.162097,-0.205391,1.000000,-0.741647,0.017562,0.003803,0.014451,0.005666,-0.008616,...,-0.008742,-0.022524,-0.004275,0.015597,-0.009794,-0.001066,0.002305,0.004947,0.014911,0.012088
Component5_fraction,0.028830,0.056293,0.063267,-0.741647,1.000000,0.000784,-0.004848,-0.004726,-0.004142,-0.007248,...,0.023205,0.002005,-0.019091,-0.007568,0.010096,0.011797,0.009477,0.007961,0.010008,0.005447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Component3_Visc_Index,0.006799,0.000670,-0.012119,-0.001066,0.011797,-0.032115,0.005463,-0.032231,0.051967,-0.003773,...,-0.006205,-0.025820,0.000603,-0.018536,-0.013417,1.000000,-0.033565,-0.014322,0.026345,0.016418
Component4_Visc_Index,-0.016042,0.015175,-0.005723,0.002305,0.009477,-0.001411,0.051331,-0.012324,-0.002322,-0.002776,...,-0.007882,0.029241,0.000466,0.001560,0.024949,-0.033565,1.000000,-0.025208,0.145118,0.142020
Component5_Visc_Index,-0.038140,0.008998,0.020535,0.004947,0.007961,0.011962,0.008306,0.013595,0.021193,0.000460,...,-0.009631,0.010931,0.010468,0.017342,-0.003710,-0.014322,-0.025208,1.000000,0.022677,0.032171
Blend_Visc_Index,-0.017628,0.005702,-0.005795,0.014911,0.010008,-0.033564,-0.007880,0.000299,-0.012339,-0.038177,...,0.006121,0.096620,0.027934,0.058577,0.085858,0.026345,0.145118,0.022677,1.000000,0.910826


In [50]:
test_dataset.corr()

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
Component1_fraction,1.000000,-0.417048,-0.471631,-0.167389,-0.008025,-0.027875,0.053371,-0.033673,0.021528,-0.036265,...,0.087875,-0.015246,8.067531e-03,-0.022745,-0.051726,-0.027253,0.016165,-0.031384,0.051916,0.053481
Component2_fraction,-0.417048,1.000000,-0.414109,-0.232240,0.127492,-0.066247,-0.036146,-0.030215,-0.055298,0.030308,...,-0.021001,-0.068870,-1.798708e-02,0.025552,-0.027303,-0.047138,0.002990,-0.012047,-0.000568,-0.008005
Component3_fraction,-0.471631,-0.414109,1.000000,-0.127871,0.002758,0.022666,-0.008772,0.052068,-0.016428,0.013606,...,-0.057357,0.089360,4.088864e-02,0.001234,0.038046,0.071599,-0.079797,0.058767,-0.037559,-0.028843
Component4_fraction,-0.167389,-0.232240,-0.127871,1.000000,-0.693851,0.082520,-0.042324,0.017643,0.066503,-0.013267,...,-0.044324,0.005887,-2.999864e-02,0.015851,0.079788,0.006494,0.043194,-0.010713,-0.024702,-0.023615
Component5_fraction,-0.008025,0.127492,0.002758,-0.693851,1.000000,0.000474,0.053983,-0.006125,-0.017857,0.011192,...,0.053872,-0.023382,-1.216407e-02,-0.034740,-0.055724,-0.006029,0.048707,-0.011495,0.012440,0.004121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Component3_Visc_Index,-0.027253,-0.047138,0.071599,0.006494,-0.006029,0.041829,0.072967,0.013896,0.062971,-0.039238,...,-0.012524,0.002293,1.024779e-02,0.059567,-0.031880,1.000000,-0.067017,-0.062365,-0.008237,-0.007775
Component4_Visc_Index,0.016165,0.002990,-0.079797,0.043194,0.048707,-0.038648,0.077484,-0.010318,-0.023415,-0.041951,...,0.003464,-0.081022,-1.330528e-02,-0.005346,-0.009518,-0.067017,1.000000,-0.002016,0.224228,0.198419
Component5_Visc_Index,-0.031384,-0.012047,0.058767,-0.010713,-0.011495,0.007695,0.027681,-0.027988,-0.015058,-0.003946,...,-0.219701,0.069831,-1.696028e-02,0.063523,0.047106,-0.062365,-0.002016,1.000000,-0.065482,-0.053600
Blend_Visc_Index,0.051916,-0.000568,-0.037559,-0.024702,0.012440,-0.080831,-0.028490,-0.043929,-0.027459,0.006363,...,0.003366,0.001392,-1.893496e-16,-0.041048,-0.013691,-0.008237,0.224228,-0.065482,1.000000,0.981551


In [51]:
print(list(train_dataset.isna().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [52]:
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# train_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# poly = PolynomialFeatures(degree=(2),interaction_only=True,include_bias=False)
# poly.fit(train_dataset)

In [53]:
# from sklearn.preprocessing import StandardScaler

In [54]:
# train_dataset = poly.transform(train_dataset)
# scaler = StandardScaler().fit(train_dataset)
# train_dataset = scaler.transform(train_dataset)

In [55]:
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_dataset = poly.transform(test_dataset)
# test_dataset = scaler.transform(test_dataset)

In [56]:
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')
# train_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# train_dataset=train_dataset.interpolate(method='linear',limit_direction='both')
# poly = PolynomialFeatures(degree=(2),include_bias=False)
# poly.fit(train_dataset.iloc[:,:55])
# train_log = np.log1p(np.maximum(train_dataset.iloc[:, :55], 1e-6))  

# train_dataset_55 = poly.transform(train_dataset.iloc[:,:55])
# train_dataset = np.hstack([train_dataset_55, train_log,  np.array(train_dataset.iloc[:,55:])])
# scaler = StandardScaler().fit(train_dataset)
# train_dataset = pd.DataFrame(scaler.transform(train_dataset))

In [57]:
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)
# test_dataset=test_dataset.interpolate(method='linear',limit_direction='both')#.isna().sum()
# test_log = np.log1p(np.maximum(test_dataset.iloc[:, :55], 1e-6))

# test_dataset_55 = poly.transform(test_dataset.iloc[:,:55])
# test_dataset = pd.DataFrame(np.hstack([test_dataset_55, test_log, np.array(test_dataset.iloc[:,55:])]))
# test_dataset = pd.DataFrame(scaler.transform(test_dataset))

In [58]:
# from sklearn.preprocessing import StandardScaler

# # Fix infinities and NaNs
# train_dataset = train_dataset.replace([np.inf, -np.inf], np.nan)
# train_dataset = train_dataset.interpolate(method='linear', limit_direction='both')
# train_dataset = train_dataset.fillna(train_dataset.mean())

# test_dataset = test_dataset.replace([np.inf, -np.inf], np.nan)
# test_dataset = test_dataset.interpolate(method='linear', limit_direction='both')
# test_dataset = test_dataset.fillna(test_dataset.mean())

# # Apply log transform only to relevant features
# log_cols = [col for col in train_dataset.columns if 'fraction' in col or 'Property' in col]
# for col in log_cols:
#     train_dataset[col] = np.log1p(np.maximum(train_dataset[col], 1e-6))
#     test_dataset[col]  = np.log1p(np.maximum(test_dataset[col], 1e-6))

# # Normalize everything
# scaler = StandardScaler()
# train_dataset = pd.DataFrame(scaler.fit_transform(train_dataset), columns=train_dataset.columns)
# test_dataset  = pd.DataFrame(scaler.transform(test_dataset), columns=test_dataset.columns)


In [59]:
from sklearn.feature_selection import VarianceThreshold

# --- 1. Drop near-zero variance features ---
# Threshold is the variance; tune as needed (e.g. 1e-5)
var_selector = VarianceThreshold(threshold=1e-5)
var_selector.fit(train_dataset)

# Keep the mask of good features
mask = var_selector.get_support()
selected_features = train_dataset.columns[mask]

# Reduce both datasets to those features
train_dataset = train_dataset[selected_features]
test_dataset  = test_dataset[selected_features]

# --- 2. Drop highly correlated features ---
# Compute correlation matrix on train only
corr_matrix = train_dataset.corr().abs()
upper_tri  = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# Identify columns to drop (corr > 0.98)
to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.98)]

# Drop them
train_dataset = train_dataset.drop(columns=to_drop)
test_dataset  = test_dataset.drop(columns=to_drop)

print(f"Dropped {len(to_drop)} correlated features: {to_drop}")
print(f"Final feature count: {train_dataset.shape[1]}")


Dropped 75 correlated features: ['Component4_fraction_Sq', 'Component4_fraction_Sqrt', 'Comp1_Prop1_Adj', 'Comp2_Prop1_Adj', 'Comp3_Prop1_Adj', 'Comp4_Prop1_Adj', 'Comp5_Prop1_Adj', 'Comp1_Prop2_Adj', 'Comp2_Prop2_Adj', 'Comp3_Prop2_Adj', 'Comp4_Prop2_Adj', 'Comp5_Prop2_Adj', 'Comp1_Prop3_Adj', 'Comp2_Prop3_Adj', 'Comp3_Prop3_Adj', 'Comp4_Prop3_Adj', 'Comp5_Prop3_Adj', 'Comp1_Prop4_Adj', 'Comp2_Prop4_Adj', 'Comp3_Prop4_Adj', 'Comp4_Prop4_Adj', 'Comp5_Prop4_Adj', 'Comp1_Prop5_Adj', 'Comp2_Prop5_Adj', 'Comp3_Prop5_Adj', 'Comp4_Prop5_Adj', 'Comp5_Prop5_Adj', 'Comp1_Prop6_Adj', 'Comp2_Prop6_Adj', 'Comp3_Prop6_Adj', 'Comp4_Prop6_Adj', 'Comp5_Prop6_Adj', 'Comp1_Prop7_Adj', 'Comp2_Prop7_Adj', 'Comp3_Prop7_Adj', 'Comp4_Prop7_Adj', 'Comp5_Prop7_Adj', 'Comp1_Prop8_Adj', 'Comp2_Prop8_Adj', 'Comp3_Prop8_Adj', 'Comp4_Prop8_Adj', 'Comp5_Prop8_Adj', 'Comp1_Prop9_Adj', 'Comp2_Prop9_Adj', 'Comp3_Prop9_Adj', 'Comp4_Prop9_Adj', 'Comp5_Prop9_Adj', 'Comp1_Prop10_Adj', 'Comp2_Prop10_Adj', 'Comp3_Prop10_Adj'

In [60]:
test_dataset

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component4_Cetane_Index,Component5_Cetane_Index,Blend_Cetane,Component1_Visc_Index,Component2_Visc_Index,Component3_Visc_Index,Component4_Visc_Index,Component5_Visc_Index,Blend_Visc_Index,Blend_Viscosity
ID,,,,,,,,,,,,,,,,,,,,,
1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,0.602809,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.299825,-1.303419,0.549224
2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,-1.943463,...,430.534316,423.240164,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-1.219901,-1.303419,0.549224
3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,-0.294728,...,533.739728,663.815825,476.367101,-1.055064,-0.845686,-0.663087,-1.111906,-0.970853,-1.303419,0.549224
4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,-0.830186,...,533.739728,517.803098,476.367101,-1.055064,-2.890751,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,0.756849,...,456.068496,424.246233,476.367101,-1.055064,-0.845686,-1.197669,-3.376006,-0.970853,-1.303419,0.549224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,0.395524,...,422.428886,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224
497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,1.032029,...,436.733566,517.803098,476.367101,-1.055064,-0.845686,0.012447,-1.111906,-1.866185,-1.303419,0.549224
498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,1.032029,...,533.739728,517.803098,476.367101,-1.055064,-0.845686,-1.197669,-1.111906,-0.970853,-1.303419,0.549224


In [61]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 483 entries, Component1_fraction to Blend_Viscosity
dtypes: float64(433), int64(50)
memory usage: 7.4 MB


## Splitting the  test data and make testset and validationset

In [62]:
train_X, val_X, train_y, val_y = train_test_split(train_dataset,output_blends, random_state=42, test_size=0.001,)

In [63]:
# sns.pairplot(train_dataset)
print("Train Shape:", train_dataset.shape)
print("Test Shape :", test_dataset.shape)
print("Any NaNs:", train_dataset.isna().sum().sum(), test_dataset.isna().sum().sum())
print("Any Infs:", np.isinf(train_dataset.to_numpy()).sum(), np.isinf(test_dataset.to_numpy()).sum())


Train Shape: (2000, 483)
Test Shape : (500, 483)
Any NaNs: 0 0
Any Infs: 0 0


# Use Tablenet The transformet approach

In [38]:
!pip install tabpfn

In [64]:
from tabpfn import TabPFNClassifier, TabPFNRegressor


In [65]:
tab_reg = TabPFNRegressor(device="auto",random_state=42,n_jobs=12)
multioutput_tab_reg = MultiOutputRegressor(estimator=tab_reg)

In [136]:
multioutput_tab_reg.fit(train_X, train_y)

MultiOutputRegressor(estimator=TabPFNRegressor(n_jobs=12, random_state=42))

In [138]:
import joblib

# Save to file
joblib.dump(multioutput_tab_reg, 'multioutput_tabpfn_model_v2.pkl')


['multioutput_tabpfn_model.pkl']

In [66]:
# Load from file
import joblib

loaded_model = joblib.load('multioutput_tabpfn_model.pkl')

# Now you can use loaded_model.predict(...)
# preds = loaded_model.predict(val_X)


In [45]:
tab_reg_pred = preds # loaded_model.predict(val_X)
print("MSE: ", mean_squared_error(val_y, tab_reg_pred))
print("MAE: ", mean_absolute_error(val_y, tab_reg_pred))
print("R2 : ", r2_score(val_y, tab_reg_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, tab_reg_pred))


MSE:  0.0047077895142138
MAE:  0.037359997630119324
R2 :  0.9953439831733704
MAPE: 0.09279724210500717


In [67]:
tab_reg_pred = loaded_model.predict(test_dataset)
tab_reg_pred = pd.DataFrame(tab_reg_pred,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
tab_reg_pred.to_csv('_output_tabpfreg.csv')
tab_reg_pred

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
ID,,,,,,,,,,
1,0.146256,0.228081,0.724894,0.613146,0.353818,0.705358,0.687902,0.360928,-0.336821,0.347265
2,-0.792640,-0.610718,-1.197447,0.063722,-0.728111,-0.120152,-1.191815,-1.050328,-0.933696,0.030162
3,1.761511,1.128711,1.062112,1.069652,2.471631,1.876843,1.027547,1.999289,0.784179,2.252076
4,-0.457512,0.309946,0.901764,-0.718745,1.878562,-0.453220,0.831742,1.781024,0.877961,-0.963787
5,0.158982,-1.220972,1.099539,0.456752,2.454814,0.276937,1.068825,-0.131095,-0.505778,1.060656
...,...,...,...,...,...,...,...,...,...,...
496,0.173990,-0.857383,1.234391,-0.290468,-0.278197,-0.743100,1.197358,-0.444928,-1.457120,-0.432238
497,-2.176230,-1.322172,-1.020217,-2.378759,-0.627009,-2.445888,-1.020925,-1.907284,-1.337142,-1.347159
498,1.999039,2.208122,0.303988,1.188128,0.007698,0.677275,0.290452,0.999529,0.230334,0.483903


In [ ]:
import pandas as pd

from autogluon.tabular import TabularPredictor


if __name__ == '__main__':  
    train_data = pd.read_csv('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
    subsample_size = 5000
    if subsample_size is not None and subsample_size < len(train_data):
        train_data = train_data.sample(n=subsample_size, random_state=0)
    test_data = pd.read_csv('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')

    tabpfnmix_default = {
        "model_path_classifier": "autogluon/tabpfn-mix-1.0-classifier",
        "model_path_regressor": "autogluon/tabpfn-mix-1.0-regressor",
        "n_ensembles": 1,
        "max_epochs": 30,
    }

    hyperparameters = {
        "TABPFNMIX": [
            tabpfnmix_default,
        ],
    }

    label = "age"
    problem_type = "regression"

    predictor = TabularPredictor(
        label=label,
        problem_type=problem_type,
    )
    predictor = predictor.fit(
        train_data=train_data,
        hyperparameters=hyperparameters,
        verbosity=3,
    )

    predictor.leaderboard(test_data, display=True)


In [34]:
# {'estimator__colsample_bytree': 0.982627653632069, 'estimator__learning_rate': 0.1822102743060054, 'estimator__max_bin': 43, 'estimator__max_depth': 17, 'estimator__max_leaves': 15, 'estimator__n_estimators': 94, 'estimator__reg_alpha': 0.10781820337059697, 'estimator__reg_lambda': 0.31170074035318485}
# reg_alpha

In [102]:
from scipy.stats import randint, uniform
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error
from xgboost import XGBRegressor

# 1) define your base XGB
xgbr = XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    device='cuda',
)

# 2) wrap it
multi_output_model_xgbr = MultiOutputRegressor(estimator=xgbr)

# 3) define your scorer
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# 4) tune using estimator__… prefixes
param_distributions = {
    'estimator__n_estimators': randint(50, 500),
    'estimator__max_depth': randint(5, 50),
    'estimator__learning_rate': uniform(0.001, 0.2),
    'estimator__gamma': uniform(0, 1),
    'estimator__min_child_weight': randint(1, 20),
    'estimator__subsample': uniform(0.5, 1.0),
    'estimator__colsample_bytree': uniform(0.3, 1.0),
    'estimator__reg_alpha': uniform(0, 1),
    'estimator__reg_lambda': uniform(0.1, 5),
}

searcher_xgbr = RandomizedSearchCV(
    multi_output_model_xgbr,
    param_distributions,
    n_iter=40,
    scoring=scorer,
    n_jobs=12,
    cv=3,
    refit=True,
    random_state=42,
)


searcher_xgbr.fit(train_X, train_y)

print("Best hyper-parameters:", searcher.best_params_)
best_model_xgbr = searcher.best_estimator_

# 6) evaluate
xgb_pred = best_model_xgbr.predict(val_X)
print("MSE: ", mean_squared_error(val_y, xgb_pred))
print("MAE: ", mean_absolute_error(val_y, xgb_pred))
print("R2 : ", r2_score(val_y, xgb_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, xgb_pred))


/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:32:13] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1738880503067/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:32:14] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1738880503067/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. 

Best hyper-parameters: {'estimator__colsample_bytree': np.float64(0.4996737821583597), 'estimator__gamma': np.float64(0.5142344384136116), 'estimator__learning_rate': np.float64(0.1194829137724085), 'estimator__max_depth': 7, 'estimator__min_child_weight': 5, 'estimator__n_estimators': 356, 'estimator__reg_alpha': np.float64(0.6803075385877797), 'estimator__reg_lambda': np.float64(2.352496259847715), 'estimator__subsample': np.float64(0.5132649611598665)}


/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:38:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1738880503067/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


MSE:  0.046059124171733856
MAE:  0.1534205824136734
R2 :  0.9558683633804321
MAPE: 0.5641891360282898


In [106]:
print(search.best_params_)
print("MSE: ", mean_squared_error(val_y, xgb_pred))
print("MAE: ", mean_absolute_error(val_y, xgb_pred))
print("R2 : ", r2_score(val_y, xgb_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, xgb_pred))

# {'estimator__colsample_bytree': np.float64(0.5619286433204139), 'estimator__learning_rate': np.float64(0.14791684291885698), 'estimator__max_bin': 136, 'estimator__max_depth': 286, 'estimator__max_leaves': 30, 'estimator__n_estimators': 649, 'estimator__reg_alpha': np.float64(0.2679130529974323), 'estimator__reg_lambda': np.float64(0.90330309864098)}
# MSE:  0.02499525621533394
# MAE:  0.12181054055690765
# R2 :  0.6237972974777222
# MAPE: 0.8065568208694458

{'estimator__reg__alpha': 100, 'estimator__pca__n_components': 0.95}
MSE:  0.046059124171733856
MAE:  0.1534205824136734
R2 :  0.9558683633804321
MAPE: 0.5641891360282898


In [104]:
test_predxgb = best_model_xgbr.predict(test_dataset)
test_predxgb = pd.DataFrame(test_predxgb,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_predxgb.to_csv('_output_xgb.csv')
test_predxgb

In [105]:
test_predxgb

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,-0.174386,0.168390,0.574824,0.508574,0.335412,0.673315,0.654097,0.601905,-0.050899,0.418690
1,-0.505634,-0.291975,-1.113956,0.104029,-0.664771,0.025051,-1.077803,-1.117453,-0.725424,0.028268
2,1.388782,0.920783,0.835106,1.159357,2.180149,1.551313,0.807961,1.715932,0.441363,2.072835
3,-0.277880,0.543634,0.476634,-0.507923,1.954679,-0.171681,0.438586,1.495245,0.520599,-0.794743
4,-0.038244,-1.530675,1.058816,0.181177,2.480243,0.208828,1.081871,-0.243285,-0.561468,0.802150
...,...,...,...,...,...,...,...,...,...,...
496,-1.844237,-0.674361,-0.889244,-2.013194,-0.641881,-2.066451,-0.851695,-1.759861,-0.926196,-1.172233
497,1.992274,1.928920,0.309796,1.108681,0.021985,0.630445,0.278604,1.048996,0.162869,0.309459
498,0.047433,0.379243,1.329445,-0.690162,-0.887947,-0.031279,1.393493,0.728338,-0.169068,1.122767
499,-0.999500,-1.782081,-2.022702,-1.577939,-0.051163,-1.699045,-1.976742,-1.807518,-2.136424,-0.122491


In [172]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression

# 1) build a single-output pipeline
pipeline_lr = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    VarianceThreshold(threshold=0.0),
    SelectKBest(score_func=f_regression, k=200),
    StandardScaler(),
    LinearRegression(fit_intercept=True, copy_X=True, n_jobs=12)
)
# 2) wrap in MultiOutputRegressor
multi_output_lr = MultiOutputRegressor(pipeline_lr)

# 3) scorer
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# 4) param grid must reference the pipeline *inside* the wrapper:
param_distributions = {
    'estimator__simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'estimator__selectkbest__k': [50, 100, 150, 200, 'all'],
    'estimator__linearregression__fit_intercept': [True, False],
    'estimator__linearregression__copy_X': [True, False],
}


# 5) randomized search
searcher_lr = RandomizedSearchCV(
    estimator=multi_output_lr,
    param_distributions=param_distributions,
    n_iter=50,
    scoring=scorer,
    cv=5,
    n_jobs=12,
    verbose=2,
    refit=True,
    random_state=42
)

# 6) fit on train
searcher_lr.fit(train_X, train_y)

# 7) results
print("Best parameters:", searcher_lr.best_params_)
best_lr = searcher_lr.best_estimator_
# best_lr = multi_output_lr.fit(train_X,train_y)
# 8) evaluate on hold‑out
lr_pred = best_lr.predict(val_X)
print("MSE :", mean_squared_error(val_y, lr_pred))
print("MAE :", mean_absolute_error(val_y, lr_pred))
print("R2  :", r2_score(val_y, lr_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, lr_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters: {'estimator__simpleimputer__strategy': 'most_frequent', 'estimator__selectkbest__k': 100, 'estimator__linearregression__fit_intercept': True, 'estimator__linearregression__copy_X': True}
MSE : 0.021403494582098195
MAE : 0.07746999338075829
R2  : 0.9820143862905587
MAPE: 0.15887336200416502


In [109]:
print("Best parameters:", searcher_lr.best_params_)
print("MSE :", mean_squared_error(val_y, lr_pred))
print("MAE :", mean_absolute_error(val_y, lr_pred))
print("R2  :", r2_score(val_y, lr_pred))
print("MAPE:", mean_absolute_percentage_error(val_y, lr_pred))
# Best parameters: {'estimator__simpleimputer__strategy': 'mean', 'estimator__linearregression__fit_intercept': False, 'estimator__linearregression__copy_X': True}
# MSE : 0.06548908205715132
# MAE : 0.11981774831745469
# R2  : 0.6284936932298283
# MAPE: 0.2448504736913187

Best parameters: {'estimator__simpleimputer__strategy': 'most_frequent', 'estimator__selectkbest__k': 100, 'estimator__linearregression__fit_intercept': True, 'estimator__linearregression__copy_X': True}
MSE : 0.021403494582098195
MAE : 0.07746999338075829
R2  : 0.9820143862905587
MAPE: 0.15887336200416502


In [110]:
test_predlr = best_lr.predict(test_dataset)
test_predlr = pd.DataFrame(test_predlr,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_predlr.to_csv('output_lr_1.csv')

In [112]:
test_predlr

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10
0,0.233170,0.056089,0.685334,0.610805,-0.243113,0.713691,0.626097,0.371923,-0.315364,0.345720
1,-0.771739,-0.514899,-1.229224,0.083303,-0.758975,-0.103732,-1.226383,-1.036513,-0.714904,0.011865
2,1.680019,1.187073,1.316651,1.129596,1.761939,1.861125,1.341925,2.027144,0.471088,2.248216
3,-0.360468,0.347118,0.700941,-0.657280,1.845324,-0.439139,0.672291,1.586482,0.410468,-0.964350
4,0.060649,-1.306055,1.004533,0.432282,2.665202,0.237373,1.014212,-0.018129,-0.199160,1.018489
...,...,...,...,...,...,...,...,...,...,...
496,-2.106839,-1.313324,-0.956853,-2.398588,-0.845990,-2.460332,-0.908410,-2.154043,-1.271468,-1.363481
497,1.948256,2.359023,0.329978,1.244118,0.430851,0.660721,0.329959,1.058901,0.306584,0.455262
498,-0.103597,0.604660,1.426056,-1.334314,-0.704433,0.177603,1.543871,0.807801,0.077436,1.261492
499,-1.111294,-1.881909,-2.090544,-1.648825,0.194676,-1.818294,-2.068892,-1.950099,-2.142446,-0.227986


In [122]:
pipeline_rid = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    VarianceThreshold(),        # drop zero‐ or near‐zero‐variance features
    PCA(),        
    Ridge(
        solver='auto',
        random_state=42,
        alpha=np.float64(0.08066305219717405),
    )
)
param_distributions_rid = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'variancethreshold__threshold': [0.0, 1e-3, 1e-2],              # test dropping features with var < threshold
    'pca__n_components': [0.90, 0.95, 0.99, None],                  # keep 90%,95%,99% variance (None ⇒ skip PCA)
    'pca__svd_solver': ['auto', 'full'],                            # solver for PCA
    'ridge__alpha': uniform(0.01, 20),                              # widen alpha range
    'ridge__solver': ['auto', 'sag', 'saga', 'lsqr'],               # focus on solvers that scale well
}

searcher_rid = RandomizedSearchCV(
    estimator=pipeline_rid,
    param_distributions=param_distributions_rid,
    n_iter=50,               
    scoring='neg_mean_absolute_percentage_error',  
    cv=3,
    n_jobs=12,
    verbose=1,
    # random_state=42,
    refit=True
)


search_rid = searcher_rid.fit(train_X, train_y)
print("Best params:", search_rid.best_params_)
best_ridge_pipeline = search_rid.best_estimator_
# best_ridge_pipeline=pipeline_rid.fit(train_X,train_y)
rid_pred = best_ridge_pipeline.predict(val_X)

print("MSE:  ", mean_squared_error(val_y, rid_pred))
print("MAE:  ", mean_absolute_error(val_y, rid_pred))
print("R2:   ", r2_score(val_y, rid_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rid_pred))


Fitting 3 folds for each of 50 candidates, totalling 150 fits


/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best params: {'pca__n_components': None, 'pca__svd_solver': 'auto', 'ridge__alpha': np.float64(0.8695363731325279), 'ridge__solver': 'lsqr', 'simpleimputer__strategy': 'most_frequent', 'variancethreshold__threshold': 0.0}
MSE:   0.030743863672270656
MAE:   0.08992072519369894
R2:    0.9745349269682337
MAPE:  0.22898910661404087


In [123]:
print("Best params:", search_rid.best_params_)
print("MSE:  ", mean_squared_error(val_y, rid_pred))
print("MAE:  ", mean_absolute_error(val_y, rid_pred))
print("R2:   ", r2_score(val_y, rid_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rid_pred))

Best params: {'pca__n_components': None, 'pca__svd_solver': 'auto', 'ridge__alpha': np.float64(0.8695363731325279), 'ridge__solver': 'lsqr', 'simpleimputer__strategy': 'most_frequent', 'variancethreshold__threshold': 0.0}
MSE:   0.030743863672270656
MAE:   0.08992072519369894
R2:    0.9745349269682337
MAPE:  0.22898910661404087


In [124]:
test_pred_rid = best_ridge_pipeline.predict(test_dataset)
test_pred_rid = pd.DataFrame(test_pred_rid,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_rid.to_csv('_output_rid.csv')

In [167]:
# import tensorflow as tf
# from tensorflow.keras import layers, models, regularizers, callbacks
# from tensorflow.keras.optimizers import AdamW

# def build_strong_tabular_model(
#     n_features=579, 
#     n_targets=10,
#     embed_dim=32,
#     transformer_heads=4,
#     transformer_ff=128,
#     depth_residual=3,
#     l2=1e-4,
#     dropout=0.3
# ):
#     inp = layers.Input(shape=(n_features,), name="features")
#     # 1) Per‑feature embedding
#     x = layers.Reshape((n_features, 1))(inp)
#     x = layers.Conv1D(embed_dim, 1, 
#                       kernel_regularizer=regularizers.l2(l2),
#                       name="feature_embedding")(x)

#     # 2) Single Transformer encoder block
#     attn = layers.MultiHeadAttention(transformer_heads, embed_dim,
#                                      kernel_regularizer=regularizers.l2(l2),
#                                      name="mha")(x, x)
#     attn = layers.Dropout(dropout)(attn)
#     attn = layers.Add()([x, attn])
#     attn = layers.LayerNormalization()(attn)

#     ff = layers.Dense(transformer_ff, activation="relu",
#                       kernel_regularizer=regularizers.l2(l2))(attn)
#     ff = layers.Dropout(dropout)(ff)
#     ff = layers.Dense(embed_dim,
#                       kernel_regularizer=regularizers.l2(l2))(ff)
#     x = layers.Add()([attn, ff])
#     x = layers.LayerNormalization(name="transformer_out")(x)

#     # 3) Flatten
#     x = layers.Flatten()(x)  # → (batch, 579*32)

#     # 4) Residual Dense blocks with projection of shortcut → 256 dims
#     for i in range(depth_residual):
#         # project the shortcut to 256 dims
#         shortcut = layers.Dense(
#             256, activation=None,
#             kernel_regularizer=regularizers.l2(l2),
#             name=f"shortcut_proj_{i}"
#         )(x)
        
#         x = layers.Dense(
#             256, activation="relu",
#             kernel_regularizer=regularizers.l2(l2),
#             name=f"res_dense_{i}_1"
#         )(x)
#         x = layers.Dropout(dropout)(x)
#         x = layers.Dense(
#             256, activation=None,
#             kernel_regularizer=regularizers.l2(l2),
#             name=f"res_dense_{i}_2"
#         )(x)
        
#         x = layers.Add(name=f"res_add_{i}")([shortcut, x])
#         x = layers.LayerNormalization(name=f"res_norm_{i}")(x)

#     # 5) Final head
#     x = layers.Dense(
#         128, activation="relu",
#         kernel_regularizer=regularizers.l2(l2),
#         name="head_dense"
#     )(x)
#     x = layers.Dropout(dropout)(x)
#     outputs = layers.Dense(
#         n_targets, activation="linear",
#         name="outputs"
#     )(x)

#     model = models.Model(inputs=inp, outputs=outputs, name="StrongTabNet")
#     model.compile(
#         optimizer=AdamW(learning_rate=3e-4, weight_decay=1e-5),
#         loss="mean_absolute_percentage_error",
#         metrics=[
#             tf.keras.metrics.MeanAbsoluteError(name="mae"),
#             tf.keras.metrics.RootMeanSquaredError(name="rmse"),
#             tf.keras.metrics.MeanAbsolutePercentageError(name='mean_absolute_percentage_error')
#         ]
#     )
#     return model

# # --- Usage ---
# model = build_strong_tabular_model(
#     n_features=579,
#     n_targets=10,
#     embed_dim=16,
#     transformer_heads=1,
#     transformer_ff=128,
#     depth_residual=1,
#     l2=1e-4,
#     dropout=0.4
# )
# model.summary()




In [168]:
# history = model.fit(
#     train_X, train_y,
#     validation_data=(val_X, val_y),
#     epochs=200,
#     batch_size=32,
#     callbacks=[
#         callbacks.EarlyStopping(patience=10, restore_best_weights=True),
#         callbacks.ReduceLROnPlateau(factor=0.5, patience=10, min_lr=1e-6)
#     ]
# )

In [169]:
# mlp_pred = model.predict(val_X)

In [170]:
# print("MSE:  ", mean_squared_error(val_y,  mlp_pred))
# print("MAE:  ", mean_absolute_error(val_y,  mlp_pred))
# print("R2:   ", r2_score(val_y,mlp_pred))
# print("MAPE: ", mean_absolute_percentage_error(val_y, mlp_pred))

In [171]:
# test_pred_mlp = model.predict(test_dataset)
# test_pred_mlp = np.expm1(test_pred_mlp)
# test_pred_mlp = pd.DataFrame(test_pred_mlp,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
#        'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
#        'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
# test_pred_mlp.to_csv('_output_mlp.csv')

In [ ]:
pipeline_rf = make_pipeline(
    SimpleImputer(strategy='mean'),
    RandomForestRegressor(    # baki hai
        n_estimators=378,
        criterion='squared_error',
        max_depth=232,
        min_samples_split=218,
        min_samples_leaf=498,
        max_features='log2',
        bootstrap=True,
        n_jobs=12,
        random_state=42,
        verbose=0,
        warm_start=False,
        max_leaf_nodes= 155,
    )
)

# Best RF params: {'randomforestregressor__criterion': 'squared_error', 'randomforestregressor__max_depth': 232, 'randomforestregressor__max_features': 'log2', 'randomforestregressor__max_leaf_nodes': 155, 'randomforestregressor__min_samples_leaf': 498, 'randomforestregressor__min_samples_split': 218, 'randomforestregressor__n_estimators': 378, 'simpleimputer__strategy': 'mean'}
# MSE:   1.1939122283517236
# MAE:   0.8203986469907789
# R²:    -10.916110902599737
# MAPE:  1.04675604732512

param_distributions_rf = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'randomforestregressor__criterion': ["squared_error", "absolute_error", "friedman_mse", "poisson"],
    'randomforestregressor__n_estimators': randint(3, 1000),
    'randomforestregressor__min_samples_split': randint(2, 500),
    'randomforestregressor__min_samples_leaf': randint(1, 500),
    'randomforestregressor__max_features': ["sqrt", "log2", None],
    'randomforestregressor__max_leaf_nodes': randint(3, 1000),
    'randomforestregressor__max_depth': randint(3, 800),
}

searcher_rf = RandomizedSearchCV(
    estimator=pipeline_rf,
    param_distributions=param_distributions_rf,
    n_iter=10,
    scoring='neg_mean_absolute_percentage_error',  # minimize MAPE
    cv=3,
    n_jobs=-1,
    verbose=2, 
    # random_state=42,
    refit=True
)

search_rf = searcher_rf.fit(train_X, train_y)

print("Best RF params:", search_rf.best_params_)

best_rf = search_rf.best_estimator_
# best_rf = pipeline_rf.fit(train_X,train_y)
rf_pred = best_rf.predict(val_X)

# 8) Evaluation metrics
print("MSE:  ", mean_squared_error(val_y, rf_pred))
print("MAE:  ", mean_absolute_error(val_y, rf_pred))
print("R²:   ", r2_score(val_y, rf_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rf_pred))


In [ ]:
print("Best RF params:", search_rf.best_params_)
print("MSE:  ", mean_squared_error(val_y, rf_pred))
print("MAE:  ", mean_absolute_error(val_y, rf_pred))
print("R²:   ", r2_score(val_y, rf_pred))
print("MAPE: ", mean_absolute_percentage_error(val_y, rf_pred))

In [ ]:
test_pred_rf = best_rf.predict(test_dataset)
test_pred_mlp = pd.DataFrame(test_pred_rf,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_rf.to_csv('_output_rf.csv')

In [ ]:
pipeline_etc = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    ExtraTreesRegressor(
        n_estimators=100,
        criterion='squared_error',
        max_depth=515,
        min_samples_split=217,
        min_samples_leaf=226,
        max_features='log2',
        bootstrap=True,
        random_state=42,
        ccp_alpha = np.float64(0.4911411839915161),
        n_jobs=12,
        warm_start=False
    )
)
# Best ExtraTrees params: {'extratreesregressor__bootstrap': True, 'extratreesregressor__ccp_alpha': np.float64(0.4911411839915161), 'extratreesregressor__max_depth': 515, 'extratreesregressor__max_features': 'log2', 'extratreesregressor__min_samples_leaf': 226, 'extratreesregressor__min_samples_split': 217, 'extratreesregressor__n_estimators': 100, 'simpleimputer__strategy': 'most_frequent'}
# MSE:   1.3215288190857684
# MAE:   0.869117328594581
# R²:    -13.578031179018907
# MAPE:  0.9793419452393474

param_distributions_etc = {
    'simpleimputer__strategy': ['mean', 'median', 'most_frequent'],
    'extratreesregressor__n_estimators': randint(5, 800),
    'extratreesregressor__max_depth': [None] + list(range(5, 800, 5)),
    'extratreesregressor__min_samples_split': randint(2, 500),
    'extratreesregressor__min_samples_leaf': randint(1, 500),
    'extratreesregressor__max_features': ['sqrt', 'log2', None] + list(uniform(0.1, 0.9).rvs(5)),
    'extratreesregressor__bootstrap': [False, True],
    'extratreesregressor__ccp_alpha': uniform(0.0, 0.9),  # cost-complexity pruning
}

searcher_etc = RandomizedSearchCV(
    estimator=pipeline_etc,
    param_distributions=param_distributions_etc,
    n_iter=100,
    scoring='neg_mean_absolute_percentage_error',  # minimize MAPE
    cv=5,
    n_jobs=12,
    verbose=0,
    random_state=42,
    refit=True
)

search_etc = searcher_etc.fit(train_X, train_y)

print("Best ExtraTrees params:", search_etc.best_params_)

best_etc = search_etc.best_estimator_
# best_etc = pipeline_etc.fit(train_X,train_y)
etc_pred = best_etc.predict(val_X)

# 8) Evaluation metrics
print("MSE:  ", mean_squared_error(val_y, etc_pred))
print("MAE:  ", mean_absolute_error(val_y, etc_pred))
print("R²:   ", r2_score(val_y, etc_pred))
print("MAPE: ", mean_absolute_percentage_erinteraction_only=Falseror(val_y, etc_pred))


In [ ]:
print("Best ExtraTrees params:", search_etc.best_params_)
print("MSE:  ", mean_squared_error(val_y, etc_pred))
print("MAE:  ", mean_absolute_error(val_y, etc_pred))
print("R²:   ", r2_score(val_y, etc_pred))
print("MAPE: ", mean_abs

In [ ]:
test_pred_etc = best_etc.predict(test_dataset)
test_pred_etc = pd.DataFrame(test_pred_etc,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_etc.to_csv('_output_etc.csv')

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import StackingRegressor

# 1) single‐output stacking (each estimator here must be single‐output)
base_estimators = [
    ('xgb', xgbr),
    ('lr',  pipeline_lr),
    ('mlp', pipeline_mlp),
    # ('rf',  pipeline_rf),
    ('etc', pipeline_etc),
]

stack_single = StackingRegressor(
    estimators=base_estimators,
    final_estimator=pipeline_rid,  # pipeline_rid is SimpleImputer+Ridge
    cv=5,
    n_jobs=-1
)

# 2) wrap the entire stack
stack_multi = MultiOutputRegressor(stack_single)

# 3) fit & predict
stack_multi.fit(train_X, train_y)       # train_y shape = (n_samples, n_targets)
y_pred_stack = stack_multi.predict(val_X)


In [ ]:
print("MSE :", mean_squared_error(val_y, y_pred_stack))
print("MAE :", mean_absolute_error(val_y, y_pred_stack))
print("R2  :", r2_score(val_y, y_pred_stack))
print("MAPE:", mean_absolute_percentage_error(val_y, y_pred_stack))


In [ ]:
test_pred_rid = stack_multi.predict(test_dataset)
test_pred_rid = pd.DataFrame(test_pred_rid,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_rid.to_csv('output_check_1_poly.csv')

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# Define the pipeline
from sklearn.preprocessing import FunctionTransformer
pipeline_svr = make_pipeline(
    SimpleImputer(strategy= 'median'),
    SVR(
        C = 0.1,
    )
)


multioutput_svr = MultiOutputRegressor(pipeline_svr)

# Correct parameter grid with proper prefix
param_distributions_svr = {
    'estimator__simpleimputer__strategy': ['mean','median'],
    'estimator__standardscaler': [StandardScaler()],
    'estimator__svr__kernel': ['rbf','linear','poly'],
    'estimator__svr__C': [0.1, 1, 10, 100],
    'estimator__svr__gamma': ['scale', 0.01, 0.1, 1],
    'estimator__svr__epsilon': [0.01, 0.1, 0.5, 1],
    'estimator__svr__degree': [2,3,4]  # only used if kernel='poly'
}

searcher_svr = RandomizedSearchCV(
    estimator=multioutput_svr,
    param_distributions=param_distributions_svr,
    n_iter=50,
    scoring='neg_mean_absolute_percentage_error',  # minimize MAPE
    cv=3,
    n_jobs=-1,
    verbose=2,
    random_state=42,
    refit=True
)

# # search_svr = searcher_svr.fit(train_X, train_y)

# print("Best SVR params:", search_svr.best_params_)

# best_svr = search_svr.best_estimator_
# # best_svr = multioutput_svr.fit(train_X,train_y)
# svr_pred = best_svr.predict(val_X)

# # Evaluation metrics
# print("MSE:  ", mean_squared_error(val_y, svr_pred))
# print("MAE:  ", mean_absolute_error(val_y, svr_pred))
# print("R²:   ", r2_score(val_y, svr_pred))
# print("MAPE: ", mean_absolute_percentage_error(val_y, svr_pred))


In [ ]:
# !pip install catboost

In [ ]:
# from catboost import CatBoostRegressor


# pipeline_cat = make_pipeline(
#     SimpleImputer(),
#     MultiOutputRegressor(
#         CatBoostRegressor(
#             iterations=1000,
#             learning_rate=0.001,
#             depth=6,
#             silent=True,
#             random_state=42,
#             thread_count=-1,
#             # task_type='GPU',
#             # devices='0',
#             # bootstrap_type='Bayesian'  # Default for GPU, robust
#         )
#     )
# )

# param_distributions_cat = {
#     'simpleimputer__strategy': ['mean', 'median'],
#     'multioutputregressor__estimator__iterations': randint(100, 2000),
#     'multioutputregressor__estimator__learning_rate': uniform(0.01, 0.3),
#     'multioutputregressor__estimator__depth': randint(4, 10),
#     'multioutputregressor__estimator__l2_leaf_reg': uniform(1, 10),
#     'multioutputregressor__estimator__rsm': uniform(0.5, 0.5),
#     'multioutputregressor__estimator__border_count': randint(32, 255),
#     # 'multioutputregressor__estimator__subsample': uniform(0.5, 0.5),  # REMOVE THIS LINE
# }

# searcher_cat = RandomizedSearchCV(
#     estimator=pipeline_cat,
#     param_distributions=param_distributions_cat,
#     n_iter=10,
#     scoring='neg_mean_absolute_percentage_error',
#     cv=3,
#     n_jobs=-1,
#     verbose=2,
#     random_state=42,
#     refit=True
# )

# # search_cat = searcher_cat.fit(train_X, train_y)

# # print("Best CatBoost params:", search_cat.best_params_)

# # best_cat = search_cat.best_estimator_
# # cat_pred = best_cat.predict(val_X)

# # print("MSE:  ", mean_squared_error(val_y, cat_pred))
# # print("MAE:  ", mean_absolute_error(val_y, cat_pred))
# # print("R²:   ", r2_score(val_y, cat_pred))
# # print("MAPE: ", mean_absolute_percentage_error(val_y, cat_pred))


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from scipy.stats import randint, uniform

# Build pipeline
pipeline_gbm = make_pipeline(
    SimpleImputer(),  # fill missing values
    GradientBoostingRegressor(
        loss='squared_error',
        learning_rate=0.001,
        n_estimators=300,
        subsample=1.0,
        min_samples_split=10,
        min_samples_leaf=20,
        min_weight_fraction_leaf=0.0,
        max_depth=10,
        min_impurity_decrease=0.0,
        init=None,
        random_state=None,
        max_features=None,
        alpha=0.99,
        verbose=2,
        max_leaf_nodes=500,
        warm_start=True,
        validation_fraction=0.1,
        n_iter_no_change=None,
        tol=0.0001,
        ccp_alpha=0.0,
    )
)

# Wrap in multioutput
multioutput_gbm = MultiOutputRegressor(pipeline_gbm)

# Parameter grid
param_distributions_gbm = {
    'estimator__simpleimputer__strategy': ['mean', 'median'],
    'estimator__gradientboostingregressor__n_estimators': randint(50, 500),
    'estimator__gradientboostingregressor__learning_rate': uniform(0.01, 0.3),
    'estimator__gradientboostingregressor__max_depth': randint(3, 10),
    'estimator__gradientboostingregressor__subsample': uniform(0.5, 0.5),
    'estimator__gradientboostingregressor__max_features': [1.0, 'sqrt', 'log2', None],
    'estimator__gradientboostingregressor__min_samples_split': randint(2, 20),
    'estimator__gradientboostingregressor__min_samples_leaf': randint(1, 20),
}


# Search
searcher_gbm = RandomizedSearchCV(
    estimator=multioutput_gbm,
    param_distributions=param_distributions_gbm,
    n_iter=50,
    scoring='neg_mean_absolute_percentage_error',
    cv=3,
    n_jobs=-1,
    verbose=0,
    # random_state=42,
    refit=True
)

# search_gbm = searcher_gbm.fit(train_X, train_y)

# # Results
# print("Best GBM params:", search_gbm.best_params_)

# best_gbm = search_gbm.best_estimator_
# gbm_pred = best_gbm.predict(val_X)

# print("MSE:  ", mean_squared_error(val_y, gbm_pred))
# print("MAE:  ", mean_absolute_error(val_y, gbm_pred))
# print("R²:   ", r2_score(val_y, gbm_pred))
# print("MAPE: ", mean_absolute_percentage_error(val_y, gbm_pred))


In [ ]:
train_y

In [56]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import StackingRegressor

# 1) single‐output stacking (each estimator here must be single‐output)
base_estimators = [
    ('xgb', xgbr),
    ('lr',  pipeline_lr),
    ('mlp', pipeline_mlp),
    ('rf',  pipeline_rf),
    ('etc', pipeline_etc),
    ('gbm', pipeline_gbm),
]

stack_single = StackingRegressor(
    estimators=base_estimators,
    final_estimator=pipeline_rid,  # pipeline_rid is SimpleImputer+Ridge
    cv=5,
    n_jobs=-1
)

# 2) wrap the entire stack
stack_multi = MultiOutputRegressor(stack_single)

# 3) fit & predict
stack_multi.fit(train_X, train_y)       # train_y shape = (n_samples, n_targets)
y_pred_stack = stack_multi.predict(val_X)

/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


      Iter       Train Loss   Remaining Time 
         1           0.9850           15.17m
         2           0.9832           14.46m
         3           0.9814           13.95m
         4           0.9796           13.68m
         5           0.9778           13.48m
         6           0.9760           13.29m
         7           0.9743           13.00m
         8           0.9725           12.81m
         9           0.9707           12.58m
        10           0.9690           12.39m
        11           0.9672           12.24m
        12           0.9655           12.15m
        13           0.9637           12.04m
        14           0.9620           11.93m
        15           0.9602           11.82m
        16           0.9585           11.71m
        17           0.9567           11.60m
        18           0.9550           11.52m
        19           0.9533           11.43m
        20           0.9515           11.33m
        21           0.9498           11.24m
        2

[14:58:09] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1738880503067/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.



       182           0.7113            4.39m
       183           0.7100            4.35m
       184           0.7088            4.32m
       185           0.7075            4.28m
       186           0.7063            4.24m
       187           0.7050            4.20m
       188           0.7038            4.16m
       189           0.7025            4.12m
       190           0.7012            4.08m
       191           0.7000            4.04m
       192           0.6987            4.00m
       193           0.6975            3.96m
       194           0.6962            3.93m
       195           0.6950            3.89m
       196           0.6938            3.85m
       197           0.6925            3.81m
       198           0.6913            3.77m
       199           0.6900            3.73m
       200           0.6888            3.69m
       201           0.6876            3.66m
       202           0.6864            3.62m
       203           0.6851            3.58m
       204

/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
 

      Iter       Train Loss   Remaining Time 
      Iter       Train Loss   Remaining Time 
      Iter       Train Loss   Remaining Time 
      Iter       Train Loss   Remaining Time 
      Iter       Train Loss   Remaining Time 
         1           0.9666           17.54m
         1           0.9564           17.60m
         1           1.0000           17.80m
         1           0.9993           18.60m
         1           1.0017           19.07m
         2           0.9982           16.65m
         2           0.9547           16.60m
         2           0.9649           16.81m
         2           0.9999           17.46m
         2           0.9975           17.55m
         3           0.9964           16.08m
         3           0.9530           16.14m
         3           0.9632           16.29m
         3           0.9957           17.01m
         3           0.9981           17.11m
         4           0.9512           15.98m
         4           0.9615           16.10m
     

/home/harshit/miniconda3/envs/dl_env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


       290           0.6000           20.72s
       288           0.5766           25.05s
       288           0.6023           25.08s
       294           0.5950           12.30s
       295           0.5753           10.22s
       291           0.5989           18.65s
       289           0.5756           22.96s
       289           0.6012           22.98s
       295           0.5940           10.25s
       296           0.5743            8.17s
       292           0.5979           16.57s
       290           0.5746           20.87s
       290           0.6002           20.89s
       296           0.5929            8.20s
       297           0.5733            6.13s
       293           0.5968           14.50s
       291           0.5736           18.78s
       291           0.5991           18.80s
       297           0.5919            6.15s
       294           0.5958           12.42s
       298           0.5723            4.09s
       292           0.5726           16.69s
       292

KeyboardInterrupt: 

In [ ]:
print("MSE :", mean_squared_error(val_y, y_pred_stack))
print("MAE :", mean_absolute_error(val_y, y_pred_stack))
print("R2  :", r2_score(val_y, y_pred_stack))
print("MAPE:", mean_absolute_percentage_error(val_y, y_pred_stack))

test_pred_rid = stack_multi.predict(test_dataset)
test_pred_rid = pd.DataFrame(test_pred_rid,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
       'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
       'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
test_pred_rid.to_csv('output_check_2_poly.csv')

In [ ]:
# from sklearn.ensemble import VotingRegressor
# from sklearn.multioutput import MultiOutputRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# # Ensure your pipelines are single-output (not wrapped in MultiOutput)
# estimators = [
#     ('xgb', xgbr),
#     ('rf',  pipeline_rf),
#     ('etc', pipeline_etc),
#     ('mlp', pipeline_mlp),
#     ('lr',  pipeline_lr),       # single-output pipeline
#     ('rid', pipeline_rid),      # single-output pipeline
#     ('gbm', pipeline_gbm),      # single-output pipeline
# ]

# base_voting = VotingRegressor(estimators=estimators, n_jobs=-1)
# voting = MultiOutputRegressor(base_voting, n_jobs=-1)

# voting.fit(train_X, train_y)

# voting_pred = voting.predict(val_X)
# print("Voting MSE:  ", mean_squared_error(val_y, voting_pred))
# print("Voting MAE:  ", mean_absolute_error(val_y, voting_pred))
# print("Voting R²:   ", r2_score(val_y, voting_pred))
# print("Voting MAPE: ", mean_absolute_percentage_error(val_y, voting_pred))


In [ ]:
# print("MSE :", mean_squared_error(val_y, voting_pred))
# print("MAE :", mean_absolute_error(val_y, voting_pred))
# print("R2  :", r2_score(val_y, voting_pred))
# print("MAPE:", mean_absolute_percentage_error(val_y, voting_pred))

# test_pred_voting = voting.predict(test_dataset)
# test_pred_voting = pd.DataFrame(test_pred_voting,columns=['BlendProperty1', 'BlendProperty2', 'BlendProperty3', 'BlendProperty4',
#        'BlendProperty5', 'BlendProperty6', 'BlendProperty7', 'BlendProperty8',
#        'BlendProperty9', 'BlendProperty10'],index=test_dataset.index)
# test_pred_voting.to_csv('output_check_3.csv')

# Applying deeplearning